In [1]:
import torch
import numpy as np
import os
from torch import optim
import torch.nn.functional as F
import random
from PIL import Image

# 数据预处理
ROOT_PATH = 'D:\\数据库\\小样本数据库\\mini-imagenet\\mini-imagenet'

import torchvision.transforms as transforms
from PIL import Image

img_size=30

# 生成数据集的x，y

In [2]:
import os.path as osp
from PIL import Image

from torch.utils.data import Dataset
from torchvision import transforms

class MiniImageNet(Dataset):

    def __init__(self, setname):
        csv_path = osp.join(ROOT_PATH, setname + '.csv')
        #filename                 label
        #n0153282900000005.jpg    n01532829
        lines = [x.strip() for x in open(csv_path, 'r').readlines()][1:]

        imgs = []  #图片
        labels = []  #图片对应的类别索引
        lb = -1

        self.wnids = []  #记录该数据集都有哪些类，每个元素（类名）的索引对应label
        
        self.transform = transforms.Compose([  #图片转换
            transforms.Resize(img_size),
            transforms.CenterCrop(img_size),  #84
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

        for l in lines:
            name, wnid = l.split(',')
            #path = osp.join(ROOT_PATH, 'images', name)  #每张图片的地址
            img=self.transform(Image.open(osp.join(ROOT_PATH, 'images', name)).convert('RGB'))
            if wnid not in self.wnids:
                self.wnids.append(wnid)
                lb += 1
            imgs.append(img)
            labels.append(lb)

        self.imgs = imgs
        self.labels = labels
        self.classes, self.counts = np.unique(self.labels, return_counts=True)
        self.classes = torch.LongTensor(self.classes)
        #print(self.classes)
        
        #print(len(self.data))  #训练集一共38400个样本，测试集12000，验证集9600
        #print(len(self.wnids)) #训练集包含64个类别，测试集20类，验证集16类，每类有600个样本
        #print(self.data[0])  #self.data包含每个样本图片的地址  D:\数据库\小样本数据库\mini-imagenet\mini-imagenet\images\n0153282900000005.jpg
        #print(self.label[0])  #self.label包含每个样本的类别 [0~63]   

    def __len__(self):
        return len(self.imgs)
        
    def __getitem__(self, idx):
        img0_tuple=torch.randperm(len(self.classes))[0]   #第一个样本随机选择一类
        should_get_same_class = random.randint(0,1)  #随机决定是否要获取同类样本
        if should_get_same_class:  
            img1_tuple = img0_tuple
        else:
            while True:    
                #直到找到非同一类别
                img1_tuple = torch.randperm(len(self.classes))[0]   
                if img0_tuple !=img1_tuple:
                    break
            
        img0_idx = torch.randperm(int(self.counts[img0_tuple]))[0] #取出图片一
        img1_idx = torch.randperm(int(self.counts[img1_tuple]))[0] #取出图片二
        
        idx0=img0_tuple*600+img0_idx
        idx1=img1_tuple*600+img1_idx

        return self.imgs[idx0], self.imgs[idx1], should_get_same_class

In [9]:
def dataloader(mode,batch_size):
    dataset=MiniImageNet(mode)
    return torch.utils.data.DataLoader(dataset, shuffle=True,batch_size=batch_size)

train_loader=dataloader('train',10)
val_loader=dataloader('val',20)
test_loader=dataloader('test',20)
#next(iter(train_loader))[0].shape  #([32, 1, 28, 28])


tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])


# 构造孪生网络模型

In [4]:
import torch.nn as nn

class SiamsNet(nn.Module):
    def __init__(self):
        super(SiamsNet, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),   #padding
            nn.Conv2d(3, 4, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(4),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(4, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),
        )

        self.fc1 = nn.Sequential(
            nn.Linear(8*30*30, 500),
            nn.ReLU(inplace=True),
            
            nn.Linear(500, 500),
            nn.ReLU(inplace=True),
            
            nn.Linear(500, 5))

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)  #拉直
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2
    
#自定义损失函数
class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [5]:
model = SiamsNet().cuda() #定义模型且移至GPU
criterion = ContrastiveLoss() #定义损失函数
optimizer = optim.Adam(model.parameters(), lr = 0.0005) #定义优化器

# 训练模型

In [6]:
counter = []
loss_history = [] 
iteration_number = 0


#开始训练
for epoch in range(0, 2):
    for i, data in enumerate(train_loader, 0):
        img0, img1 , label = data
        #img0维度为torch.Size([32, 1, 100, 100])，32是batch，label为torch.Size([32, 1])
        img0, img1, label=img0.type(torch.FloatTensor),img1.type(torch.FloatTensor),label.type(torch.FloatTensor),
        img0, img1 , label = img0.cuda(), img1.cuda(), label.cuda() #数据移至GPU
        optimizer.zero_grad()
        output1,output2 = model(img0, img1)
        loss_contrastive = criterion(output1, output2, label)
        loss_contrastive.backward()
        optimizer.step()
        if i % 10 == 0 :
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())
    print("Epoch number: {} , Current loss: {:.4f}\n".format(epoch,loss_contrastive.item()))
    

Epoch number: 0 , Current loss: 1.0928

Epoch number: 1 , Current loss: 0.9500



In [7]:
from tqdm import tqdm
import random
    
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_loss = [] #记录每个eposide，每个任务更新的结果

val_loss = []

best_loss = 0
best_model_path='C:/Users/user/Desktop/最近/best_siamsnet/best_model_m.pth'
last_model_path='C:/Users/user/Desktop/最近/best_siamsnet/last_model_m.pth'

#遍历epoch训练
for epoch in range(5):
    print('=== Epoch: {} ==='.format(epoch))
    #取一个batch的训练数据
    tr_iter = iter(train_loader)
    model.train()
    for eposide in tqdm(tr_iter):  #遍历每个eposide，一共遍历iterations次，#len(next(iter(dataloader))) #[75个图片矩阵，75个标签]
        
        x1, x2, y = eposide   #x.shape=([75, 1, 28, 28])
        #print(x1.shape)
        x1, x2, y=x1.type(torch.FloatTensor),x2.type(torch.FloatTensor),y.type(torch.FloatTensor),
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        optimizer.zero_grad()  #梯度清零
        
        output1,output2= model(x1,x2)  #前传
        loss= criterion(output1,output2, y)  #计算loss
 
        loss.backward()  #反传
        optimizer.step()  #参数更新
        
        train_loss.append(loss.item())   #加入这个batch的计算loss
        
    avg_loss = np.mean(train_loss[-100:])  #计算本次epoch的所有batch迭代的平均损失
    print('Avg Train Loss: {}'.format(avg_loss))
    
    #每训练一个batch=iterations个任务后，验证一次模型
    val_iter = iter(val_loader)
    model.eval()
    for eposide in val_iter:
        x1, x2, y = eposide   #x.shape=([75, 1, 28, 28])
        x1, x2, y=x1.type(torch.FloatTensor),x2.type(torch.FloatTensor),y.type(torch.FloatTensor),
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        
        output1,output2= model(x1,x2)
        loss= criterion(output1,output2, y)  #计算loss
        
        val_loss.append(loss.item())

    avg_loss = np.mean(val_loss[-100:])

    #如果当前模型的验证效果比先前好，则把当前模型记为最佳模型
    postfix = ' (Best)' if avg_loss <= best_loss else ' (Best: {})'.format(best_loss)
    print('Avg Val Loss: {},{}'.format(  #输出本次验证的平均损失和准确率
            avg_loss, postfix))

    #保存某个epoch内所有batch在验证集上得到的最佳模型、准确率
    if avg_loss <= best_loss:
        torch.save(model.state_dict(), best_model_path)
        best_loss = avg_loss
        best_state = model.state_dict()

#print(len(train_loss))  #iterations
#print(len(val_loss)) #iterations

#保存最终所有epoch得到的最佳模型
torch.save(model.state_dict(), last_model_path)
print('最后的结果： best_loss=%d, train_loss=%d, val_loss=%d' % (best_loss, train_loss[-1] ,val_loss[-1]))


  0%|                                                                                 | 2/3840 [00:00<03:15, 19.62it/s]

=== Epoch: 0 ===


100%|██████████████████████████████████████████████████████████████████████████████| 3840/3840 [03:03<00:00, 20.91it/s]


Avg Train Loss: 1.0891497021913528


  0%|                                                                                 | 2/3840 [00:00<03:12, 19.90it/s]

...

Avg Train Loss: 1.1005493062734604


  0%|                                                                                 | 2/3840 [00:00<03:17, 19.40it/s]

Avg Val Loss: 1.0852788001298905, (Best: 0)
=== Epoch: 4 ===


100%|██████████████████████████████████████████████████████████████████████████████| 3840/3840 [03:02<00:00, 21.06it/s]


Avg Train Loss: 1.0896085071563721
Avg Val Loss: 1.0708659887313843, (Best: 0)
最后的结果： best_loss=0, train_loss=0, val_loss=1


# 测试模型

In [10]:
test_loss = list()
for epoch in range(10):
    test_iter = iter(val_loader)
    for eposide in test_iter:
        x1, x2, y = eposide   #x.shape=([75, 1, 28, 28])
        x1, x2, y=x1.type(torch.FloatTensor),x2.type(torch.FloatTensor),y.type(torch.FloatTensor),
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        
        output1,output2= model(x1,x2)
        loss= criterion(output1,output2, y)  #计算loss
        
        test_loss.append(loss.item())
        
avg_loss = np.mean(test_loss)
print('Test loss: {}'.format(avg_loss))

Test loss: 1.08528855368495


In [11]:
model=None
#加载训练最好的模型参数
model = SiamsNet().to(device)
model.load_state_dict(torch.load(os.path.join('C:\\Users\\user\\Desktop\\最近\\best_siamsnet','last_model_m.pth')))
#model.load_state_dict(best_state)
print('Testing with best model..')

#测试模型
test_loss = list()
for epoch in range(10):
    test_iter = iter(val_loader)
    for eposide in test_iter:
        x1, x2, y = eposide   #x.shape=([75, 1, 28, 28])
        x1, x2, y=x1.type(torch.FloatTensor),x2.type(torch.FloatTensor),y.type(torch.FloatTensor),
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        
        output1,output2= model(x1,x2)
        loss= criterion(output1,output2, y)  #计算loss
        
        test_loss.append(loss.item())
        
avg_loss = np.mean(test_loss)
print('Test loss: {}'.format(avg_loss))

Testing with best model..
Test loss: 1.0861465167750914
